In [12]:
import numpy as np

In [1]:
def Random_Motif_Matrix(Dna,k):
    motif_matrix = np.zeros(len(Dna)).astype('str')
    for i in range(len(Dna)):
        init_pos = np.random.randint(len(Dna[i])-k+1)
        motif_matrix[i] = Dna[i][init_pos:init_pos+k]
    return motif_matrix

In [2]:
def form_Profile(Motifs,k):
    ####Version1
    #Frequence = {base:np.zeros(k) for base in ['A','C','G','T']}
    #Sum = np.zeros(k)
    ####Version 2 with pseudocounts
    Frequence = {base:np.ones(k) for base in ['A','C','G','T']}
    Sum = np.ones(k)      
    for motif in Motifs:
        for i in range(len(motif)):
            Frequence[motif[i]][i] +=1
            Sum[i] += 1
    
    Profile = {base:np.zeros(k) for base in ['A','C','G','T']}
    for base,val in Profile.items():
        for i in range(len(val)):
            val[i] = Frequence[base][i]/Sum[i]
    return Profile

In [98]:
def Profile_randomly_generated_kmer(Text,k,Profile):
    n = len(Text)-k+1
    probs = np.ones(n) 
    Kmers = np.ones(n).astype('str')
    for i in range(n):
        kmer = Text[i:i+k]
        Kmers[i] = kmer
        for j in range(len(kmer)):
            base = kmer[j]
            probs[i] *= Profile[base][j]
    kmer = np.random.choice(Kmers,1,p=probs/np.sum(probs))[0]
    return kmer

In [100]:
def Score(Motifs,k):
    Frequence = {pos:{base:0 for base in ['A','C','G','T']} for pos in range(k)}
    Sum = np.zeros(k)
    for motif in Motifs:
        for i in range(len(motif)):
            Frequence[i][motif[i]] +=1
            Sum[i] += 1
    Score = 0
    for i in range(k):
        max_base = max(Frequence[i], key=Frequence[i].get)
        Score += Sum[i] - Frequence[i][max_base]
    return Score

In [108]:
def GibbsSampler(Dna, k, t, N):
    Motifs = Random_Motif_Matrix(Dna,k)
    BestMotifs = Motifs.copy()
    for j in range(N):
        i = np.random.randint(t)
        Motifs_except_i = np.delete(Motifs, i)
        Profile = form_Profile(Motifs_except_i,k)
        Motifs[i]  = Profile_randomly_generated_kmer(Dna[i],k,Profile)
        if Score(Motifs,k) < Score(BestMotifs,k):
            BestMotifs = Motifs.copy()
    return BestMotifs

In [129]:
k = 8 
t = 5
N = 1000
Dna = [
'CGCCCCTCTCGGGGGTGTTCAGTAACCGGCCA',
'GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG',
'TAGTACCGAGACCGAAAGAAGTATACAGGCGT',
'TAGATCAAGTTTCAGGTGCACGTCGGTGAACC',
'AATCCACCAGCTCCACGTGCAATGTTGGCCTA'
]

In [131]:
BestMotifs = GibbsSampler(Dna, k, t,N)
for i in range(20):
    Motifs = GibbsSampler(Dna, k, t,N)
    if Score(Motifs,k) < Score(BestMotifs,k):
        BestMotifs  = Motifs.copy()
for kmer in BestMotifs:
    print(kmer)

TCTCGGGG
CCAAGGTG
TACAGGCG
TTCAGGTG
TCCACGTG


In [137]:
Dna = []
k = 0
t = 0
N = 0
file = open('dataset_369257_4.txt', 'r') 
for i, line in enumerate(file):
    line=line.rstrip('\n')
    if(i==0):
        (k,t,N)= line.split(' ')
        k = int(k)
        t = int(t)
        N = int(N)
    else:
        Dna.append(line)

In [136]:
BestMotifs = GibbsSampler(Dna, k, t,N)
for i in range(20):
    Motifs = GibbsSampler(Dna, k, t,N)
    if Score(Motifs,k) < Score(BestMotifs,k):
        BestMotifs  = Motifs.copy()
for kmer in BestMotifs:
    print(kmer)

TACTTTTGGCGGCTT
TATAAGATCGAGTTT
TATGACATCGAGTTT
ACTTTTATCGAGTTC
CTGTTTATCGAGTTT
TATTTGCCCGAGTTT
TATTTTATCGAGGGC
TACGATATCGAGTTT
TATTTTCCTGAGTTT
TATTTGTGCGAGTTT
TATTTTAGGCAGTTT
TATTTTATGTCGTTT
TATTGAGTCGAGTTT
AATTTTATCGAGTAG
TATTTTATCCTTTTT
TTGCTTATCGAGTTT
TATTTTATCGTACTT
TATTTTCGGGAGTTT
TATTCATTCGAGTTT
TATTTTATCGATGCT
